# Summation & LSTM Model - SciTail Dataset

The following notebook contains the implementation of the baseline summation model, and the standard LSTM model for the SciTail dataset

In [24]:
# Import the necessary libraries

import re
from numpy import array
from numpy import asarray
from numpy import zeros

import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.regularizers import L2
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Dropout, Input, LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /home/aakashj2412/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
# Helper function that cleans the input data and enumerates the labels

def extract(s):
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()

labels = {'entails\n': 0, 'contradicts': 1, 'neutral\n': 2}

In [26]:
# Function that reads data and parses data from file

def readFileData(filePath):
    with open(filePath, 'r') as f:
        inputRows = [row.split('\t') for row in f.readlines()[1:]]

    inputPremises = [row[0] for row in inputRows if row[2] in labels]
    inputHypotheses = [row[1] for row in inputRows if row[2] in labels]
    inputLabels = [labels[row[2]] for row in inputRows if row[2] in labels]
    f.close()

    return [inputPremises, inputHypotheses, inputLabels]

In [27]:
# Reading train and test data

datasetPath = "../../Datasets/SciTail/"
trainData = readFileData(f'{datasetPath}scitail_1.0_train.tsv')
testData = readFileData(f'{datasetPath}scitail_1.0_test.tsv')
validationData = readFileData(f'{datasetPath}scitail_1.0_dev.tsv')

In [28]:
# Hyperparameters

maxLen = 32
epochs = 100
batchSize = 512
gloveDimension = 300
hiddenDimension = 100
regularization = 4e-6

In [29]:
# Tokenizer to generate the vocabulary of the system

tokenizer = Tokenizer()
tokenizer.fit_on_texts(trainData[0] + trainData[1])
vocabSize = len(tokenizer.word_index)+1

In [30]:
# Convert the train data to sequences as per the vocabulary
trainData[0] = tokenizer.texts_to_sequences(trainData[0])
trainData[1] = tokenizer.texts_to_sequences(trainData[1])

# Pad or trim all generated sequences to the same max sentence length
trainData[0] = pad_sequences(trainData[0], maxLen, padding='post')
trainData[1] = pad_sequences(trainData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
trainData[2] = tf.keras.utils.to_categorical(trainData[2], num_classes=3)

In [31]:
# Convert the test data to sequences as per the vocabulary
testData[0] = tokenizer.texts_to_sequences(testData[0])
testData[1] = tokenizer.texts_to_sequences(testData[1])

# Pad or trim all generated sequences to the same max sentence length
testData[0] = pad_sequences(testData[0], maxLen, padding='post')
testData[1] = pad_sequences(testData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
testData[2] = tf.keras.utils.to_categorical(testData[2], num_classes=3)

In [32]:
# Convert the validation data to sequences as per the vocabulary
validationData[0] = tokenizer.texts_to_sequences(validationData[0])
validationData[1] = tokenizer.texts_to_sequences(validationData[1])

# Pad or trim all generated sequences to the same max sentence length
validationData[0] = pad_sequences(validationData[0], maxLen, padding='post')
validationData[1] = pad_sequences(validationData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
validationData[2] = tf.keras.utils.to_categorical(validationData[2], num_classes=3)

In [33]:
# Import the GloVe embeddings and generate embeddings dictionary

embeddingsDict = dict()
glovePath = '../../Datasets/GloVe/'
glove = open(f'{glovePath}glove.840B.300d.txt', encoding='utf8')

for line in glove:
    records = line.split()
    word = ''.join(records[:-300])
    vectorDimensions = asarray(records[-300:], dtype='float32')
    embeddingsDict[word] = vectorDimensions

glove.close()

In [34]:
# Iterate through the embeddings and store only those that are present in our vocabulary

embeddingsMat = zeros((vocabSize, gloveDimension))
for word, index in tokenizer.word_index.items():
    vec = embeddingsDict.get(word)
    if vec is not None:
        embeddingsMat[index] = vec

In [35]:
# Define the embedding layer for our baseline RNN model
embed = Embedding(vocabSize, gloveDimension, weights=[embeddingsMat], input_length=maxLen, trainable=False)

# As Premise and Hypothesis are distinct and are to be inputted separately, define two inputs and embed
premise = Input(shape=(maxLen,), dtype='int32')
hypothesis = Input(shape=(maxLen,), dtype='int32')

premInput = embed(premise)
hypoInput = embed(hypothesis)

convert = Dense(hiddenDimension, activation='tanh', input_shape=(gloveDimension,))

premInput = convert(premInput)
hypoInput = convert(hypoInput)

In [36]:
# Once the sentence embeddings have been generated, generate a matrix of dimensions maxLen X gloveDimension
# On adding maxLen, we get a single embedding vector of length gloveDimension

# Alternate between #1 and #2 rnn definitions to switch between summation and LSTM model

# rnn = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=hiddenDimension)
rnn = LSTM(hiddenDimension, dropout=0.2, recurrent_dropout=0.2)

In [37]:
# Apply batch normalization to the two input embeddings separately

premInput = rnn(premInput)
hypoInput = rnn(hypoInput)
premInput = tf.keras.layers.BatchNormalization()(premInput)
hypoInput = tf.keras.layers.BatchNormalization()(hypoInput)

In [38]:
# Joint is a concatenated embeddings layer, generated from the premise and hypothesis inputs
# Dilution of probability 0.2, to assist in regularization
joint = keras.layers.concatenate([premInput, hypoInput])
joint = Dropout(0.2)(joint)
for i in range(3):
    joint = Dense(2*hiddenDimension, activation='tanh', kernel_regularizer=L2(regularization))(joint)
    joint = Dropout(0.2)(joint)
    joint = tf.keras.layers.BatchNormalization()(joint)

# 3 layers of the TanH activation function, along with L2 regularization.
# The final decision is based on the Softmax function
pred = Dense(3, activation='softmax')(joint)

In [39]:
# Defining the final models input and output format, as well as compilation parameters

model = Model(inputs=[premise, hypothesis], outputs=pred)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

## Summation Model

The following subsection trains the model and reports the findings for the summation model

In [20]:
# Summation Model

callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=10)
model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=batchSize, epochs=epochs, callbacks=[callback], validation_data=[[array(validationData[0]), array(validationData[1])], array(validationData[2])])

Epoch 1/100
47/47 [==============================] - 2s 51ms/step - loss: 0.3150 - accuracy: 0.8654 - val_loss: 0.6531 - val_accuracy: 0.6969
Epoch 2/100
47/47 [==============================] - 2s 50ms/step - loss: 0.3134 - accuracy: 0.8676 - val_loss: 0.6700 - val_accuracy: 0.7084
Epoch 3/100
47/47 [==============================] - 2s 51ms/step - loss: 0.3063 - accuracy: 0.8706 - val_loss: 0.6787 - val_accuracy: 0.7153
Epoch 4/100
47/47 [==============================] - 2s 52ms/step - loss: 0.3101 - accuracy: 0.8695 - val_loss: 0.8196 - val_accuracy: 0.6677
Epoch 5/100
47/47 [==============================] - 2s 51ms/step - loss: 0.3034 - accuracy: 0.8714 - val_loss: 0.7044 - val_accuracy: 0.6915
Epoch 6/100
47/47 [==============================] - 2s 50ms/step - loss: 0.2988 - accuracy: 0.8756 - val_loss: 0.6732 - val_accuracy: 0.7053
Epoch 7/100
47/47 [==============================] - 2s 50ms/step - loss: 0.2919 - accuracy: 0.8779 - val_loss: 0.7561 - val_accuracy: 0.6938
Epoch 

In [21]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)

9/9 [==============================] - 0s 11ms/step - loss: 0.7286 - accuracy: 0.7049
Loss =  0.7286471724510193
Acc =  0.7049411535263062


In [ ]:
model.save('../../Models/summation_scitail.h5')

In [1]:
ccc = keras.models.load_model('../../Models/summation_scitail.h5')
l, a = ccc.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print(a)


39/39 [==============================] - 4s 78ms/step - loss: 0.5769 - accuracy: 0.7330
0.7730048894882202


We can thus observe the following results for the summation model, for the SciTail dataset:

- Training Accuracy: 88.21%
- Validation Accuracy: 70.84%
- Test Accuracy: 70.49%

## LSTM Model

The following subsection trains the model and reports the findings for the LSTM model

In [42]:
# LSTM Model

callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=10)
model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=batchSize, epochs=epochs, callbacks=[callback], validation_data=[[array(validationData[0]), array(validationData[1])], array(validationData[2])])


Epoch 1/100
47/47 [==============================] - 25s 388ms/step - loss: 1.1180 - accuracy: 0.5029 - val_loss: 0.8707 - val_accuracy: 0.4973
Epoch 2/100
47/47 [==============================] - 17s 372ms/step - loss: 0.7896 - accuracy: 0.6372 - val_loss: 0.8930 - val_accuracy: 0.4958
Epoch 3/100
47/47 [==============================] - 18s 379ms/step - loss: 0.6480 - accuracy: 0.6889 - val_loss: 0.7231 - val_accuracy: 0.6723
Epoch 4/100
47/47 [==============================] - 18s 375ms/step - loss: 0.5954 - accuracy: 0.7025 - val_loss: 0.6603 - val_accuracy: 0.6508
Epoch 5/100
47/47 [==============================] - 18s 373ms/step - loss: 0.5452 - accuracy: 0.7297 - val_loss: 0.6854 - val_accuracy: 0.6408
Epoch 6/100
47/47 [==============================] - 18s 378ms/step - loss: 0.5221 - accuracy: 0.7446 - val_loss: 0.7945 - val_accuracy: 0.6539
Epoch 7/100
47/47 [==============================] - 18s 385ms/step - loss: 0.4927 - accuracy: 0.7624 - val_loss: 0.9841 - val_accuracy

In [45]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)


9/9 [==============================] - 0s 42ms/step - loss: 0.9079 - accuracy: 0.6659
Loss =  0.9078975319862366
Acc =  0.7458823490142822


In [ ]:
model.save('../../Models/LSTM_scitail.h5')

We can thus observe the following results for the simple LSTM model, for the SciTail dataset:

- Training Accuracy: 89.58%
- Validation Accuracy: 72.83%
- Test Accuracy: 74.58%

In general, we observe an increase in performance by the simple LSTM model.